In [1]:
import cv2
import glob
import numpy as np

# Define color thresholds
color = ["green", "black", "white"]
color_thresholds = [3000, 20000, 45000]  # green, black, white
color_mask = [[], [], []]

key = None # Initialize key variable
images = glob.glob('test_footage/*.jpg')
images.sort() 

# Iterate through every image in the directory and check if one of the color thresholds is exceeded
#if it is, then the image will be displayed, as are the differents color filters
for filename in images:
    exceeded = False
    color_counts = [0, 0, 0]  # Initialize color counters
    # Load the image, crop and convert to YUV format
    img = cv2.imread(filename)
    img = img[:, 80:180]
    yuv_img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    
    color_mask[0] = cv2.inRange(yuv_img, (100, 0, 0), (114, 126, 155)) #found color ranges for green
    color_mask[1] = cv2.inRange(yuv_img, (40, 0, 0), (114, 126, 165)) #black
    color_mask[2] = cv2.inRange(yuv_img, (213, 0, 0), (255, 255, 255))#and white

    for i in range(3):
        color_counts[i] = np.count_nonzero(color_mask[i])
        if color_counts[i] > color_thresholds[i]:  # check color thresholds
            exceeded = True

    if exceeded:
        for i in range(3):
            print(f'{color[i]} pixels = {color_counts[i]}')
            color_mask[i] = cv2.cvtColor(color_mask[i], cv2.COLOR_GRAY2BGR)
            a = np.where((color_mask[i] == [255, 255, 255]).all(axis=2))
            color_mask[i][a] = [0, 255, 0] if i == 0 else [255, 0, 127] if i == 1 else [255, 255, 255]
            cv2.namedWindow(color[i], cv2.WINDOW_NORMAL)
            cv2.resizeWindow(color[i], 200, 750)
            cv2.imshow(color[i], color_mask[i])

        cv2.namedWindow('Original', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Original', 150, 750)
        cv2.imshow("Original", img)
        key = cv2.waitKey(0)
        cv2.destroyAllWindows()
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    # Close the image window
    cv2.destroyAllWindows()
    print(color_counts[1])
    
print(np.shape(color_mask))
print("done")

green pixels = 3935
black pixels = 16943
white pixels = 2719
(3, 520, 100, 3)
done


In [4]:
#####Extract horizon function in Python########
#Using roll and pitch variables

import cv2
import numpy as np
import glob
def extract_horizon(image, roll, pitch):
    # Convert roll and pitch angles from radians to degrees
    roll_deg = np.rad2deg(roll)
    pitch_deg = np.rad2deg(pitch)
    
    # Define image dimensions, later I willl just put the values for hight and width
    img_height, img_width = image.shape[:2]

    # Calculate the center pixel of the image, this will also be known in advance
    cx, cy = img_width // 2, img_height // 2

    # Calculate the distance from the center pixel to the top/bottom of the image based on the pitch angle
    fov = 15 #this is the field of view i.e. half of the vertical angle covered by the camera. This is an assumed value
    f = img_height / (2 * np.tan(np.deg2rad(fov)))
    dy = f * np.tan(pitch)

    # Calculate the angle between the center line and the horizon
    #not sure here either but seems to make sense
    theta = np.arctan2(dy, f)

    # Rotate the image around the center pixel by the roll angle
    M = cv2.getRotationMatrix2D((cx, cy), -roll_deg, 1)
    rotated = cv2.warpAffine(image, M, (img_width, img_height))

    # Crop the image to only show the horizon
    cropped_height = int(2 * f * np.tan(theta))
    cropped = rotated[cy - 50 : cy + 50, :]

    return cropped

image = cv2.imread("../AE4317_2019_datasets/cyberzoo_aggressive_flight/20190121-144646/32748563.jpg")


image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
roll = 0.614069
pitch = 0.068963
cropped = extract_horizon(image, roll, pitch)

cv2.imwrite("cropped.jpg", cropped)
cv2.imwrite("original.jpg", image)
cv2.imshow("Final Image", cropped)
cv2.imshow("Original", image)
cv2.waitKey(0)
cv2.destroyAllWindows()